In [1]:
sc

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1615659097244_0002,spark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

res1: org.apache.spark.SparkContext = org.apache.spark.SparkContext@7d48853d


In [279]:
import org.apache.spark.sql.{DataFrame,Column}
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types._

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

import org.apache.spark.sql.{DataFrame, Column}
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types._


In [308]:
def saveCSV(df: DataFrame, path: String, limit: Int = 15): Unit = 
(
    df
    .limit(limit)
    .repartition(1)
    .write
    .option("header", "true")
    .csv(path)
)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

saveCSV: (df: org.apache.spark.sql.DataFrame, path: String, limit: Int)Unit


In [271]:
val s3_path = "s3://trab4-bucket/datasets/debate-tweets.parquet"

val df = spark.read.parquet(s3_path)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

s3_path: String = s3://trab4-bucket/datasets/debate-tweets.parquet
df: org.apache.spark.sql.DataFrame = [Tweets: string, Date: timestamp ... 1 more field]


In [273]:
df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- Tweets: string (nullable = true)
 |-- Date: timestamp (nullable = true)
 |-- ID: long (nullable = true)



In [272]:
df.show(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-------------------+-----------+
|              Tweets|               Date|         ID|
+--------------------+-------------------+-----------+
|@justinbieber  i ...|2014-10-17 16:55:34|85899345920|
| Precisoo!!!!!☀️☀️☀️|2014-10-17 16:55:34|85899345921|
|@Silvinho95_  sau...|2014-10-17 16:55:34|85899345922|
|Que felicidad es ...|2014-10-17 16:55:35|85899345923|
|Every time i fall...|2014-10-17 16:55:34|85899345924|
|          Tenho q ri|2014-10-17 16:55:35|85899345925|
|@Maristrin @rodri...|2014-10-17 16:55:35|85899345926|
|    Los cago odiando|2014-10-17 16:55:35|85899345927|
|Só marketing ne.....|2014-10-17 16:55:35|85899345928|
|@taiis_oliveira q...|2014-10-17 16:55:35|85899345929|
+--------------------+-------------------+-----------+
only showing top 10 rows



In [274]:
val hashDF = (
    df.withColumn("splitedTweets",split(col("Tweets")," "))
    .withColumn("hashtags",filter(col("splitedTweets"),
                                  (col: Column) => col.startsWith("#"))
               )
    .withColumn("hashtags",element_at(col("hashtags"),1))
)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

hashDF: org.apache.spark.sql.DataFrame = [Tweets: string, Date: timestamp ... 3 more fields]


In [275]:
// Quantos tweets com hashtag
hashDF.filter(col("hashtags").isNotNull).count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

res375: Long = 875944


# 1 a) Quais foram as hashtags mais usadas pela manhã, tarde e noite?

In [282]:
// De manhã
val df1AManha = (
    hashDF.filter(
        col("hashtags").isNotNull
    )
    .filter(
        hour(col("Date")).between(5,12)
    )
    .groupBy("hashtags")
    .count()
    .orderBy(desc("count"))
)

saveCSV(df1AManha,"s3://trab4-bucket/datasets/questoes/1/a_manha.csv")

df1AManha.show(15,false)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------------------+-----+
|hashtags                      |count|
+------------------------------+-----+
|#EMABiggestFansJustinBieber   |23385|
|#EMABiggestFans1D             |23058|
|#VoteVampsTeenAwards          |736  |
|#trndnl                       |603  |
|#QueroNoTVZ                   |523  |
|#AustinMahone                 |492  |
|#TwOff                        |435  |
|#EMABiggestFansJustinBieber"""|423  |
|#EMABiggestFans1D"            |416  |
|#EMABiggestFansJustinBieber"  |408  |
|#EMABiggestFansJustinBieberᅠ  |401  |
|#EMABiggestFans1D"""          |353  |
|#bomdia                       |322  |
|#BomDia                       |298  |
|#NowIsGoodNaGlobo             |274  |
+------------------------------+-----+
only showing top 15 rows



In [289]:
// De tarde
val df1ATarde = (
    hashDF.filter(
        col("hashtags").isNotNull
    )
    .filter(
        hour(col("Date")).between(13,18)
    )
    .groupBy("hashtags")
    .count()
    .orderBy(desc("count"))
)

saveCSV(df1ATarde,"s3://trab4-bucket/datasets/questoes/1/a_tarde.csv")

df1ATarde.show(15,false)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

df1ATarde: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [hashtags: string, count: bigint]
+-----------------------------------+-----+
|hashtags                           |count|
+-----------------------------------+-----+
|#EMABiggestFans1D                  |59270|
|#EMABiggestFansJustinBieber        |50209|
|#StealMyGirl                       |5476 |
|#QueroNoTVZ                        |4266 |
|#bigpaynodanceoff                  |1353 |
|#EMABiggestFans1D"                 |1284 |
|#EMABiggestFansJustinBieber"""     |1283 |
|#demiyourstorydoesntdefineyou      |961  |
|#HottieOfTheWeek                   |875  |
|#EMABiggestFansJustinBieber"       |853  |
|#AustinMahone                      |788  |
|#LuanSantanaNaHoraDoFaro           |719  |
|#EMABiggestFansArianaGrande        |714  |
|#AMAs                              |693  |
|#DomingoPreguiçosoDoSDVcomValentino|678  |
+-----------------------------------+-----+
only showing top 15 rows



In [290]:
// De noite
val df1ANoite = (
    hashDF.filter(
        col("hashtags").isNotNull
    )
    .filter(
        (hour(col("Date")).between(19,23)) || (hour(col("Date")).between(0,4))
    )
    .groupBy("hashtags")
    .count()
    .orderBy(desc("count"))
)

saveCSV(df1ANoite,"s3://trab4-bucket/datasets/questoes/1/a_noite.csv")

df1ANoite.show(15,false)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

df1ANoite: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [hashtags: string, count: bigint]
+------------------------------+------+
|hashtags                      |count |
+------------------------------+------+
|#EMABiggestFans1D             |114729|
|#EMABiggestFansJustinBieber   |114277|
|#camilasayshi                 |9043  |
|#EMABiggestFansJustinBieber"""|3331  |
|#DebateNoSBT                  |2773  |
|#CartersNewVideo              |2681  |
|#bigpaynodanceoff             |2476  |
|#Vote5HEMA                    |2410  |
|#TheVoiceBrasil               |2379  |
|#LuanSantanaNaHoraDoFaro      |2191  |
|#AssistamODR                  |2123  |
|#EMABiggestFans1D"            |2102  |
|#EMABiggestFansJustinBieber"  |2039  |
|#DebateNaRecord               |1815  |
|#LinkinParkNoMultishow        |1764  |
+------------------------------+------+
only showing top 15 rows



# 1 b) Quais as hashtags mais usadas em cada dia?

In [291]:
//  15
val df1B15 =(
    hashDF.filter(
        col("hashtags").isNotNull
    )
    .filter(
        dayofmonth(col("Date")) === 15
    )
    .groupBy("hashtags")
    .count()
    .orderBy(desc("count"))
)

saveCSV(df1B15,"s3://trab4-bucket/datasets/questoes/2/b_15.csv")

df1B15.show(15,false)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

df1B15: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [hashtags: string, count: bigint]
+----------------------------------+-----+
|hashtags                          |count|
+----------------------------------+-----+
|#EMABiggestFans1D                 |29711|
|#EMABiggestFansJustinBieber       |24981|
|#StealMyGirl                      |5823 |
|#bigpaynodanceoff                 |3737 |
|#AssistamODR                      |2015 |
|#BuyLoveMeHarderOniTunes          |1276 |
|#UnlockMockingjay                 |1112 |
|#Vote5HEMA                        |1058 |
|#EMABiggestFansJustinBieber"""    |987  |
|#QueroMuitosSeguidoresComValentino|807  |
|#STEALMYGIRL                      |488  |
|#EMABiggestFansJustinBieber”      |472  |
|#EMABiggestFans5SOS               |469  |
|#EMABiggestFansJustinBieber"      |433  |
|#StealMyGirI                      |397  |
+----------------------------------+-----+
only showing top 15 rows



In [294]:
//  16
val df1B16 =(
    hashDF.filter(
        col("hashtags").isNotNull
    )
    .filter(
        dayofmonth(col("Date")) === 16
    )
    .groupBy("hashtags")
    .count()
    .orderBy(desc("count"))
)

saveCSV(df1B16,"s3://trab4-bucket/datasets/questoes/2/b_16.csv")

df1B16.show(15,false)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

df1B16: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [hashtags: string, count: bigint]
+------------------------------+-----+
|hashtags                      |count|
+------------------------------+-----+
|#EMABiggestFans1D             |63198|
|#EMABiggestFansJustinBieber   |53203|
|#camilasayshi                 |9087 |
|#DebateNoSBT                  |2689 |
|#CartersNewVideo              |2681 |
|#AustinMahone                 |1420 |
|#EMABiggestFansJustinBieber"""|1350 |
|#EMABiggestFans1D"            |1287 |
|#MasterChefBR                 |1158 |
|#EMABiggestFans1D"""          |1147 |
|#HottieOfTheWeek              |1098 |
|#EMABiggestFansJustinBieber"  |1000 |
|#askjadelittlemix             |989  |
|#LuaBlancoNoAgoraETarde       |954  |
|#camilasayhi                  |883  |
+------------------------------+-----+
only showing top 15 rows



In [295]:
//  17
val df1B17 =(
    hashDF.filter(
        col("hashtags").isNotNull
    )
    .filter(
        dayofmonth(col("Date")) === 17
    )
    .groupBy("hashtags")
    .count()
    .orderBy(desc("count"))
)

saveCSV(df1B17,"s3://trab4-bucket/datasets/questoes/2/b_17.csv")

df1B17.show(15,false)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

df1B17: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [hashtags: string, count: bigint]
+------------------------------+-----+
|hashtags                      |count|
+------------------------------+-----+
|#EMABiggestFansJustinBieber   |44672|
|#EMABiggestFans1D             |44050|
|#QueroNoTVZ                   |2868 |
|#TheVoiceBrasil               |2370 |
|#FlyNoMixDiario               |1380 |
|#EMABiggestFansJustinBieber"""|1334 |
|#LançamentoDoClipeVocêSeFoiFLY|1316 |
|#AustinMahone                 |1123 |
|#EMABiggestFans1D"            |904  |
|#HottieOfTheWeek              |769  |
|#SextaTodosSDVcomValentino    |725  |
|#EMABiggestFansJustinBieber"  |685  |
|#ChristianNoEncontroComFatima |674  |
|#AmorESexo                    |671  |
|#flyNoMixdiario               |664  |
+------------------------------+-----+
only showing top 15 rows



In [296]:
//  18
val df1B18 =(
    hashDF.filter(
        col("hashtags").isNotNull
    )
    .filter(
        dayofmonth(col("Date")) === 18
    )
    .groupBy("hashtags")
    .count()
    .orderBy(desc("count"))
)

saveCSV(df1B18,"s3://trab4-bucket/datasets/questoes/2/b_18.csv")

df1B18.show(15,false)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

df1B18: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [hashtags: string, count: bigint]
+------------------------------+-----+
|hashtags                      |count|
+------------------------------+-----+
|#EMABiggestFans1D             |25039|
|#EMABiggestFansJustinBieber   |24936|
|#QueroNoTVZ                   |2336 |
|#demiyourstorydoesntdefineyou |952  |
|#WeWantZaynsSongsInFOUR       |700  |
|#MaratonaTwilight             |672  |
|#SigaTiuMarkitoNesseSabadoSdv |592  |
|#trndnl                       |591  |
|#EMABiggestFans1D"            |570  |
|#EMABiggestFansJustinBieber"""|526  |
|#premiomultishow              |447  |
|#AMAs                         |420  |
|#MCGuimeNoRaulGil             |374  |
|#KCAArgentina                 |365  |
|#EMABiggestFansJustinBieber"  |359  |
+------------------------------+-----+
only showing top 15 rows



In [297]:
//  19
val df1B19 =(
    hashDF.filter(
        col("hashtags").isNotNull
    )
    .filter(
        dayofmonth(col("Date")) === 19
    )
    .groupBy("hashtags")
    .count()
    .orderBy(desc("count"))
)

saveCSV(df1B19,"s3://trab4-bucket/datasets/questoes/2/b_19.csv")

df1B19.show(15,false)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

df1B19: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [hashtags: string, count: bigint]
+-----------------------------------+-----+
|hashtags                           |count|
+-----------------------------------+-----+
|#EMABiggestFansJustinBieber        |30375|
|#EMABiggestFans1D                  |27948|
|#LuanSantanaNaHoraDoFaro           |2910 |
|#LinkinParkNoMultishow             |1767 |
|#demiyourstorydoesntdefineyou      |1150 |
|#VoteVampsTeenAwards               |881  |
|#ComeToBrazilMattEspinosa          |845  |
|#DomingoPreguiçosoDoSDVcomValentino|820  |
|#ChatApimentadoDoBiel              |698  |
|#EMABiggestFansJustinBieber"       |680  |
|#EMABiggestFansJustinBieber"""     |638  |
|#VamosLaU                          |616  |
|#trndnl                            |574  |
|#AcreditamosEmVoceLL               |561  |
|#FOLLOWMELOHANTHONY                |557  |
+-----------------------------------+-----+
only showing top 15 rows



In [298]:
//  20
val df1B20 =(
    hashDF.filter(
        col("hashtags").isNotNull
    )
    .filter(
        dayofmonth(col("Date")) === 20
    )
    .groupBy("hashtags")
    .count()
    .orderBy(desc("count"))
)

saveCSV(df1B20,"s3://trab4-bucket/datasets/questoes/2/b_20.csv")

df1B20.show(15,false)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

df1B20: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [hashtags: string, count: bigint]
+---------------------------+-----+
|hashtags                   |count|
+---------------------------+-----+
|#EMABiggestFansJustinBieber|9704 |
|#EMABiggestFans1D          |7111 |
|#DebateNaRecord            |1760 |
|#debatenarecord            |1556 |
|#QueroDilmaTreze           |1418 |
|#EmTodoBrasilAecio45       |638  |
|#AecioEmTodoBrasil         |607  |
|#CongratsOn1MChris         |447  |
|#Los80                     |381  |
|#PreOrderVampsUSA          |363  |
|#KCAArgentina              |320  |
|#twdstrangers              |315  |
|#austinmahonechile         |281  |
|#BuyNobodyOniTunes         |280  |
|#NoEntiendo                |247  |
+---------------------------+-----+
only showing top 15 rows



# 1 c) Qual o número de tweets por hora a cada dia?

In [309]:
//  15

val df1C15 = (
    hashDF.filter(
        dayofmonth(col("Date")) === 15
    )
    .withColumn("Hora",hour(col("Date")))
    .groupBy("Hora")
    .agg(count("Tweets"))
    .orderBy(asc("Hora"))
)

saveCSV(df1C15,"s3://trab4-bucket/datasets/questoes/1/c_15.csv",limit = 25)

df1C15.show(25,false)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

df1C15: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [Hora: int, count(Tweets): bigint]
+----+-------------+
|Hora|count(Tweets)|
+----+-------------+
|14  |34378        |
|15  |79157        |
|16  |78353        |
|17  |83950        |
|18  |77713        |
|19  |65095        |
|20  |66813        |
|21  |79270        |
|22  |86030        |
|23  |97574        |
+----+-------------+



In [310]:
//  16

val df1C16 = (
    hashDF.filter(
        dayofmonth(col("Date")) === 16
    )
    .withColumn("Hora",hour(col("Date")))
    .groupBy("Hora")
    .agg(count("Tweets"))
    .orderBy(asc("Hora"))
)

saveCSV(df1C16,"s3://trab4-bucket/datasets/questoes/1/c_16.csv",limit = 25)

df1C16.show(25,false)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

df1C16: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [Hora: int, count(Tweets): bigint]
+----+-------------+
|Hora|count(Tweets)|
+----+-------------+
|0   |110232       |
|1   |163338       |
|2   |176211       |
|3   |124599       |
|4   |77743        |
|5   |42661        |
|6   |22228        |
|7   |10157        |
|8   |8327         |
|9   |23616        |
|10  |37528        |
|11  |42716        |
|12  |49156        |
|13  |55958        |
|14  |66154        |
|15  |76891        |
|16  |79276        |
|17  |65824        |
|18  |57098        |
|19  |58547        |
|20  |68148        |
|21  |88473        |
|22  |99443        |
|23  |93350        |
+----+-------------+



In [311]:
//  17

val df1C17 = (
    hashDF.filter(
        dayofmonth(col("Date")) === 17
    )
    .withColumn("Hora",hour(col("Date")))
    .groupBy("Hora")
    .agg(count("Tweets"))
    .orderBy(asc("Hora"))
)

saveCSV(df1C17,"s3://trab4-bucket/datasets/questoes/1/c_17.csv",limit = 25)

df1C17.show(25,false)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

df1C17: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [Hora: int, count(Tweets): bigint]
+----+-------------+
|Hora|count(Tweets)|
+----+-------------+
|0   |107984       |
|1   |142770       |
|2   |153981       |
|3   |122205       |
|4   |76951        |
|5   |42222        |
|6   |20985        |
|7   |10695        |
|8   |8016         |
|9   |20559        |
|10  |35147        |
|11  |40918        |
|12  |48140        |
|13  |59851        |
|14  |66972        |
|15  |74748        |
|16  |74909        |
|17  |65007        |
|18  |58841        |
|19  |57831        |
|20  |65602        |
|21  |53146        |
|22  |53497        |
|23  |54053        |
+----+-------------+



In [312]:
//  18

val df1C18 = (
    hashDF.filter(
        dayofmonth(col("Date")) === 18
    )
    .withColumn("Hora",hour(col("Date")))
    .groupBy("Hora")
    .agg(count("Tweets"))
    .orderBy(asc("Hora"))
)

saveCSV(df1C18,"s3://trab4-bucket/datasets/questoes/1/c_18.csv",limit = 25)

df1C18.show(25,false)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

df1C18: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [Hora: int, count(Tweets): bigint]
+----+-------------+
|Hora|count(Tweets)|
+----+-------------+
|0   |55404        |
|1   |61050        |
|2   |63501        |
|3   |56562        |
|4   |42778        |
|5   |37148        |
|6   |27473        |
|7   |17724        |
|8   |12108        |
|9   |11539        |
|10  |17456        |
|11  |27491        |
|12  |41485        |
|13  |58383        |
|14  |71402        |
|15  |75206        |
|16  |72334        |
|17  |69001        |
|18  |63700        |
|19  |61727        |
|20  |59165        |
|21  |60665        |
|22  |65102        |
|23  |68663        |
+----+-------------+



In [313]:
//  19

val df1C19 = (
    hashDF.filter(
        dayofmonth(col("Date")) === 19
    )
    .withColumn("Hora",hour(col("Date")))
    .groupBy("Hora")
    .agg(count("Tweets"))
    .orderBy(asc("Hora"))
)

saveCSV(df1C19,"s3://trab4-bucket/datasets/questoes/1/c_19.csv",limit = 25)

df1C19.show(25,false)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

df1C19: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [Hora: int, count(Tweets): bigint]
+----+-------------+
|Hora|count(Tweets)|
+----+-------------+
|0   |73049        |
|1   |80626        |
|2   |89885        |
|3   |97575        |
|4   |77489        |
|5   |55958        |
|6   |40591        |
|7   |22988        |
|8   |16698        |
|9   |13707        |
|10  |15743        |
|11  |22588        |
|12  |38047        |
|13  |55888        |
|14  |69949        |
|15  |75866        |
|16  |76299        |
|17  |73534        |
|18  |80192        |
|19  |77265        |
|20  |75453        |
|21  |77033        |
|22  |86629        |
|23  |95083        |
+----+-------------+



In [314]:
//  20

val df1C20 = (
    hashDF.filter(
        dayofmonth(col("Date")) === 20
    )
    .withColumn("Hora",hour(col("Date")))
    .groupBy("Hora")
    .agg(count("Tweets"))
    .orderBy(asc("Hora"))
)

saveCSV(df1C20,"s3://trab4-bucket/datasets/questoes/1/c_20.csv",limit = 25)

df1C20.show(25,false)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

df1C20: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [Hora: int, count(Tweets): bigint]
+----+-------------+
|Hora|count(Tweets)|
+----+-------------+
|0   |123610       |
|1   |134555       |
|2   |124997       |
|3   |92015        |
|4   |57005        |
|5   |29497        |
|6   |14504        |
|7   |4922         |
+----+-------------+



# 1 d) Quais as principais sentenças relacionadas à palavra “Dilma”?

In [307]:
def splitOnLength(len: Int) = udf(
    (str: String) => {
  str.grouped(len).toSeq
})


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

splitOnLength: (len: Int)org.apache.spark.sql.expressions.UserDefinedFunction


In [315]:
val threshold = 40
val DFDilma = (
    hashDF
    .withColumn("Splited",splitOnLength(threshold)(col("Tweets")))
    .withColumn("Sentenca Dilma",
                            filter(
                                col("Splited"),
                                (col: Column) => length(col) === threshold
                            ))
    .withColumn("Sentenca Dilma",
                            filter(
                                col("Sentenca Dilma"),
                                (col: Column) => lower(col).contains("dilma")
                            ))
    .withColumn("Sentenca Dilma",element_at(col("Sentenca Dilma"),1))
    .filter(
        col("Sentenca Dilma").isNotNull && !trim(col("Sentenca Dilma")).startsWith("#")
    )
    .groupBy("Sentenca Dilma")
    .count()
    .orderBy(desc("count"))
)

saveCSV(DFDilma,"s3://trab4-bucket/datasets/questoes/1/d_dilma.csv")

DFDilma.show(15,false)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

threshold: Int = 40
DFDilma: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [Sentenca Dilma: string, count: bigint]
+----------------------------------------+-----+
|Sentenca Dilma                          |count|
+----------------------------------------+-----+
|No debate de hj, Dilma foi orientada pel|12   |
|Relatórios usados por Dilma para critica|10   |
|Dilma que fala tanto em um projeto de se|9    |
|em sobre Dilma. A previsão é que ele ven|8    |
|@os_deminas @BlogDilmaBR2 @jossimarfaria|7    |
|1. #QueroDilmaTreze 2. #EmTodoBrasilAeci|7    |
|Se eu fosse a Dilma sabendo que iria fic|7    |
|Dilma perde o rumo no meio da entrevista|7    |
|No debate Dilma Rousseff está pensando q|7    |
|Só pra lembrar: Aécio 45 Dilma 13 CPM 22|6    |
|NANCIOU CAMPANHA DE DILMA. É O MAR DE LA|6    |
|— Dilma ou Aécio? — Dois hambúrgueres, a|6    |
|"""@folha_com: Dos 22 anúncios de Dilma |5    |
|@JairoRochaFilho @JornalOGlobo @dilmabr |5    |
|"""@dilmabr: Tive uma leve queda de pres

# 1 e) Quais as principais sentenças relacionadas à palavra “Aécio”?

In [316]:
def splitOnLengthSwitchE(len: Int) = udf(
    (str: String) => {
  str.replace("aécio", "aecio").replace("Aécio", "Aecio").grouped(len).toSeq
})

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

splitOnLengthSwitchE: (len: Int)org.apache.spark.sql.expressions.UserDefinedFunction


In [317]:
val threshold = 40
val DFAecio = (
    hashDF
    .withColumn("Splited",splitOnLengthSwitchE(threshold)(col("Tweets")))
    .withColumn("Sentenca Aecio",
                            filter(
                                col("Splited"),
                                (col: Column) => length(col) === threshold
                            ))
    .withColumn("Sentenca Aecio",
                            filter(
                                col("Sentenca Aecio"),
                                (col: Column) => lower(col).contains("aecio")
                            ))
    .withColumn("Sentenca Aecio",element_at(col("Sentenca Aecio"),1))
    .filter(
        col("Sentenca Aecio").isNotNull && !trim(col("Sentenca Aecio")).startsWith("#")
    )
    .groupBy("Sentenca Aecio")
    .count()
    .orderBy(desc("count"))
)


saveCSV(DFAecio,"s3://trab4-bucket/datasets/questoes/1/d_aecio.csv")

DFAecio.show(15,false)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

threshold: Int = 40
DFAecio: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [Sentenca Aecio: string, count: bigint]
+----------------------------------------+-----+
|Sentenca Aecio                          |count|
+----------------------------------------+-----+
|r Aecio somem do site do TCE http://t.co|10   |
|Segundo pesquisa Aecio lidera com vantag|8    |
|@Prjuliocpc @MariaLuciadeJe3 @AecioNeves|8    |
| no Debate do SBT por Aecio Neves http:/|8    |
| como sendo da esposa de Aecio. Quanta p|8    |
|Pressionada por Aecio no debate do SBT, |7    |
|vo no Debate do SBT por Aecio Neves http|7    |
|Você sabia que o 'Aecio' manteve-se dura|6    |
|@neyleprevost @pricilamarikoch @AecioNev|6    |
|Aos meus amigos que votarão em Aecio htt|6    |
|O governo @AecioNeves vai levar tecnolog|6    |
|— Dilma ou Aecio? — Dois hambúrgueres, a|6    |
|@AecioNeves Mentiroso estava sentadinho |6    |
|vivo no Debate do SBT por Aecio Neves ht|6    |
|Só pra lembrar: Aecio 45 Dilma 13 CPM 22

# Rascunho

In [299]:
val dfDays = (
    hashDF.select("Date")
    .filter(
     col("hashtags").isNotNull
 )
 .withColumn("Dia",
             dayofmonth(col("Date"))
).withColumn("Mes",
             month(col("Date"))
).withColumn("Ano",
             year(col("Date"))
)
 .groupBy("Dia","Mes","Ano")
 .count()
 .orderBy(desc("count"))
)

saveCSV(dfDays,"s3://trab4-bucket/datasets/questoes/2/count_days.csv")

dfDays.show(100)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

dfDays: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [Dia: int, Mes: int ... 2 more fields]
+----+----+----+------+
| Dia| Mes| Ano| count|
+----+----+----+------+
|  16|  10|2014|236132|
|  17|  10|2014|188568|
|  19|  10|2014|158317|
|  18|  10|2014|124984|
|  15|  10|2014|113653|
|  20|  10|2014| 54274|
|null|null|null|    16|
+----+----+----+------+

